In [1]:
%matplotlib inline
import numpy as np
import time
import h5py
import keras
import pandas as pd
import math
import joblib
import matplotlib.pyplot as plt

from fuel.datasets.hdf5 import H5PYDataset

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from IPython.display import display

from keras.layers import Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, Activation, Dropout
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D

from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop, Adam, SGD
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

num_classes = 10
batch_size = 100
original_img_size = (img_rows, img_cols, 1)
epochs = 500

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1) / 255.
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1) / 255.

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [4]:
def create_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=original_img_size))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    opt = RMSprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

create_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
__________

In [5]:
results = []
for sample_size in [100, 500, 1000, 2000, 5000]:
    start = time.time()
    print('Fitting with sample_size: {}'.format(sample_size))
   
    if sample_size < len(X_train):
        sss = StratifiedShuffleSplit(n_splits=2, test_size=sample_size / len(X_train), random_state=0)
        _, index = sss.split(X_train, y_train)
        X, y = X_train[index[1]], y_train[index[1]]
    else:
        X, y = X_train, y_train
   
    y = np_utils.to_categorical(y)
    model = create_model()
    model.fit(X, y, shuffle=True, 
              epochs=epochs,
              batch_size=batch_size,
              verbose=0,
              callbacks=[EarlyStopping(monitor='loss', min_delta=0.1, patience=10)])
    
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    score = accuracy_score(y_test, y_pred)
    
    end = time.time()
    elapsed = end - start
    print(' * Accuracy: %.1f %%' % (100. * score))
    print(' * Fit time elapsed: %.1fs' % elapsed)
    results.append({'sample_size': sample_size, 'accuracy': score, 'time': elapsed})

Fitting with sample_size: 100
 * Accuracy: 26.2 %
 * Fit time elapsed: 2.0s
Fitting with sample_size: 500
 * Accuracy: 92.1 %
 * Fit time elapsed: 4.8s
Fitting with sample_size: 1000
 * Accuracy: 93.4 %
 * Fit time elapsed: 5.7s
Fitting with sample_size: 2000
 * Accuracy: 95.5 %
 * Fit time elapsed: 8.9s
Fitting with sample_size: 5000
 * Accuracy: 97.8 %
 * Fit time elapsed: 18.0s


In [6]:
df = pd.DataFrame(results)
display(df)
df.to_csv('cnn_mnist_results.csv', index=False)

,accuracy,sample_size,time
0,0.2623,100,2.039911
1,0.9213,500,4.823549
2,0.9338,1000,5.706222
3,0.9547,2000,8.887535
4,0.9780,5000,18.029773
